# Regression on the hotel reviews [sklearn]
* Regression task of predicting review 'score', based on the review text.
* Reference notebook: <https://www.kaggle.com/code/jiashenliu/simple-regression-model-most-important-words/notebook>
* Dataset: <https://www.kaggle.com/code/jiashenliu/simple-regression-model-most-important-words/input>

## Import libraries

In [ ]:
import os
from typing import Iterable

import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_extraction.text import TfidfVectorizer

from giskard import Model, Dataset, scan

## Define constants

In [ ]:
# Constants.
FEATURE_COLUMN_NAME = "Full_Review"
TARGET_COLUMN_NAME = "Reviewer_Score"

# Paths.
DATA_PATH = os.path.join("..", "datasets", "hotel_text_regression_dataset", 'Hotel_Reviews.csv')

## Load data

In [ ]:
def load_data(**kwargs) -> pd.DataFrame:
    df = pd.read_csv(DATA_PATH, **kwargs)
    df[FEATURE_COLUMN_NAME] = df.apply(lambda x: x['Positive_Review'] + ' ' + x['Negative_Review'], axis=1)
    return df

reviews_df = load_data(nrows=1000)

## Train-test split

In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split(reviews_df[[FEATURE_COLUMN_NAME]], reviews_df[TARGET_COLUMN_NAME], random_state=42)

## Wrap data with giskard

In [ ]:
raw_data = pd.concat([test_X, test_Y], axis=1)
wrapped_data = Dataset(raw_data,
                       name="hotel_text_regression_dataset",
                       target=TARGET_COLUMN_NAME,
                       column_types={FEATURE_COLUMN_NAME: "text"})

## Define preprocessing steps

In [ ]:
def adapt_vectorizer_input(df: pd.DataFrame) -> Iterable:
    """Adapt input for the vectorizers.

    The problem is that vectorizers accept iterable, not DataFrame, but Series.
    Thus, we need to ravel dataframe with text have input single dimension.
    """

    df = df.iloc[:, 0]
    return df

## Define, fit and test model

In [ ]:
# Define pipeline.
pipeline = Pipeline(steps=[
    ("vectorizer_adapter", FunctionTransformer(adapt_vectorizer_input)),
    ("vectorizer", TfidfVectorizer(max_features=10000)),
    ("regressor", GradientBoostingRegressor(n_estimators=10))
])

# Fit pipeline.
pipeline.fit(train_X, train_Y)

# Perform inference on train and test data.
pred_train = pipeline.predict(train_X)
pred_test = pipeline.predict(test_X)

In [ ]:
train_metric = mean_absolute_error(train_Y, pred_train)
test_metric = mean_absolute_error(test_Y, pred_test)

print(f"Train MAE: {train_metric: .2f}\n"
      f"Test MAE: {test_metric: .2f}")

## Define prediction function

In [ ]:
def prediction_function(df: pd.DataFrame) -> np.ndarray:
    return pipeline.predict(df)

## Wrap model with giskard

In [ ]:
wrapped_model = Model(prediction_function,
                      model_type="regression",
                      name="hotel_text_regression",
                      feature_names=[FEATURE_COLUMN_NAME])

In [ ]:
# Validate wrapped model.
pred_test_wrapped = wrapped_model.predict(wrapped_data).raw_prediction
wrapped_test_metric = mean_absolute_error(test_Y, pred_test_wrapped)
print(f"Wrapped Test MAE: {wrapped_test_metric: .2f}")

## Scan results with giskard

In [ ]:
scanning_results = scan(wrapped_model, wrapped_data)

In [ ]:
display(scanning_results)